In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import random

import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split




In [2]:
df = pd.read_csv("./daily-min-temperatures.csv")
df["year"] = [0]*len(df["Date"])
df["month"] = [0]*len(df["Date"])
df["day"] = [0]*len(df["Date"])
for i in range(len(df["year"])):
    df["year"][i] = int(df["Date"][i][0:4])
    df["month"][i] = int(df["Date"][i][5:7])
    df["day"][i] = int(df["Date"][i][8:])

<ipython-input-2-e02ed463d761>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["year"][i] = int(df["Date"][i][0:4])
<ipython-input-2-e02ed463d761>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["month"][i] = int(df["Date"][i][5:7])
<ipython-input-2-e02ed463d761>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["day"][i] = int(df["Date"][i][8:])


In [3]:
train, test = train_test_split(df, test_size= 0.2, shuffle=False)
train = train[["year", 'month', "day", "Temp"]].reset_index(drop=True)
test = test[["year", 'month', "day", "Temp"]].reset_index(drop=True)

x_train = train[["year", 'month', "day"]].values.reshape((train[["year", 'month', "day"]].values.shape[0], 1, train[["year", 'month', "day"]].values.shape[1]))
y_train = train["Temp"].values

x_test = test[["year", 'month', "day"]].values.reshape((test[["year", 'month', "day"]].values.shape[0], 1, test[["year", 'month', "day"]].values.shape[1]))
y_test = test["Temp"].values

In [4]:
#from tensorflow tutorial
class WindowGenerator():
    def __init__(self, input_width, label_width, shift,
               train_df=train[["month", "day", "Temp"]], val_df=test[["month", "day", "Temp"]], test_df=test[["month", "day","Temp"]],
               label_columns=None):
        # Store the raw data.
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df

        # Work out the label column indices.
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in
                                        enumerate(label_columns)}
        self.column_indices = {name: i for i, name in
                               enumerate(train_df.columns)}

        # Work out the window parameters.
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def __repr__(self):
        return '\n'.join([
            f'Total window size: {self.total_window_size}',
            f'Input indices: {self.input_indices}',
            f'Label indices: {self.label_indices}',
            f'Label column name(s): {self.label_columns}'])
    def split_window(self, features):
        inputs = features[:, self.input_slice, :]
        labels = features[:, self.labels_slice, :]
        if self.label_columns is not None:
            labels = tf.stack(
                [labels[:, :, self.column_indices[name]] for name in self.label_columns],
                axis=-1)

        # Slicing doesn't preserve static shape information, so set the shapes
        # manually. This way the `tf.data.Datasets` are easier to inspect.
        inputs.set_shape([None, self.input_width, None])
        labels.set_shape([None, self.label_width, None])

        return inputs, labels


    @property
    def train(self):
        return self.make_dataset(self.train_df)

    @property
    def val(self):
        return self.make_dataset(self.val_df)

    @property
    def test(self):
        return self.make_dataset(self.test_df)

    @property
    def example(self):
        """Get and cache an example batch of `inputs, labels` for plotting."""
        result = getattr(self, '_example', None)
        if result is None:
        # No example batch was found, so get one from the `.train` dataset
            result = next(iter(self.train))
        # And cache it for next time
            self._example = result
        return result

    def make_dataset(self, data):
        data = np.array(data, dtype=np.float32)
        ds = tf.keras.preprocessing.timeseries_dataset_from_array(
          data=data,
          targets=None,
          sequence_length=self.total_window_size,
          sequence_stride=1,
          shuffle=True,
          batch_size=32,)

        ds = ds.map(self.split_window)

        return ds
#     WindowGenerator.split_window = split_window
#     WindowGenerator.make_dataset = make_dataset
#     WindowGenerator.train = train
#     WindowGenerator.val = val
#     WindowGenerator.test = test
#     WindowGenerator.example = example

In [5]:
k = 14
wide_window = WindowGenerator(
    input_width=k, label_width=k, shift=1,
    label_columns=['Temp'])
wide_window

Total window size: 15
Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]
Label indices: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14]
Label column name(s): ['Temp']

In [6]:
gru_model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    layers.GRU(64, return_sequences=True),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=1)
])
gru_model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=["mse", tf.metrics.MeanAbsoluteError()])

In [7]:
gru_model.fit(wide_window.train, epochs=40, validation_data=wide_window.val)


Epoch 1/40
91/91 [==============================] - 3s 11ms/step - loss: 56.1484 - mse: 56.1484 - mean_absolute_error: 6.1134 - val_loss: 20.7196 - val_mse: 20.7196 - val_mean_absolute_error: 3.6611
Epoch 2/40
91/91 [==============================] - 1s 7ms/step - loss: 14.4861 - mse: 14.4861 - mean_absolute_error: 2.9538 - val_loss: 10.4939 - val_mse: 10.4939 - val_mean_absolute_error: 2.5153
Epoch 3/40
91/91 [==============================] - 1s 8ms/step - loss: 9.1519 - mse: 9.1519 - mean_absolute_error: 2.3163 - val_loss: 7.5316 - val_mse: 7.5316 - val_mean_absolute_error: 2.1307
Epoch 4/40
91/91 [==============================] - 1s 7ms/step - loss: 7.6352 - mse: 7.6352 - mean_absolute_error: 2.1277 - val_loss: 6.4110 - val_mse: 6.4110 - val_mean_absolute_error: 1.9739
Epoch 5/40
91/91 [==============================] - 1s 7ms/step - loss: 7.0062 - mse: 7.0062 - mean_absolute_error: 2.0445 - val_loss: 5.8875 - val_mse: 5.8875 - val_mean_absolute_error: 1.8970
Epoch 6/40
91/91 [===

In [8]:
gru_model.evaluate(wide_window.test, verbose=0)

[4.864458084106445, 4.8644585609436035, 1.7464131116867065]